In [3]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [9]:
subj = subject.Subject & {'subject_uuid': UUID('8f6fc7ce-4de0-4b8b-973b-72fd70145b9c')}
# get trial counts and session length to date
session_info = (behavior.BehavioralSummaryByDate * 
                behavior.BehavioralSummaryByDate.ReactionTimeByDate & 
                subj).proj('session_date', 'performance_easy', 'median_reaction_time').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:151: UserWarning:

Mysql server has gone away. Reconnecting to the server.



In [12]:
subj

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:151: UserWarning:

Mysql server has gone away. Reconnecting to the server.



subject_uuid,subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_source name of source,protocol_number protocol number,subject_description
8f6fc7ce-4de0-4b8b-973b-72fd70145b9c,CSHL_004,M,2018-11-20,RR,C57BL/6J,Jax,3,None


In [32]:
performance_easy = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['performance_easy'].tolist(),
    mode = 'markers+lines',
    marker = dict(
        size = 6,
        color = 'black'),
    name = 'performance easy',
    yaxis = 'y1'
)
rt = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['median_reaction_time'].tolist(),
    mode = 'markers+lines',
    marker = dict(
        size = 6,
        color = 'red'),
    name = 'reaction time',
    yaxis = 'y2'
)

data = [performance_easy, rt]

layout = go.Layout(
    yaxis=dict(
        title='Performance on easy trials',
        range=[0,1]
    ),
    yaxis2=dict(
        title='Median reaction time (s)',
        overlaying='y',
        color = 'red',
        side='right',
        type='log',
        range=np.log10([0.1, 10]).tolist(),
        dtick=np.log10([0.1, 1, 10]).tolist()
        
    ),
    xaxis=dict(
        title='Date',
    ), 
    width=500,
    height=400,
    title='Performance and median reaction time',
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [33]:
f = open("performance_reaction_time.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()